# Segmenting and Clustering Neighborhoods in Toronto Part 1

## Week 3

### 1-Scraping  Wikipedia page into a DataFrame

In [1]:
#importing libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents


In [2]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'lxml')

In [4]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.strip())
        boroughList.append(cells[1].text.strip())
        neighborhoodList.append(cells[2].text.strip()) 

### 2-Creating the DataFrame

In [5]:
# create a new DataFrame from the three lists
toronto_df = pd.DataFrame({"PostalCode": postalCodeList, "Borough": boroughList,"Neighborhood": neighborhoodList})
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### 3.1- Droping cells with a borough that is "Not assigned"

In [6]:
# drop cells with a borough that is Not assigned
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### 3.2-Grouping neighborhoods in the same borough

In [7]:
#The new version of the table in wikipedia is already grouped by postalcode, let's see how the code will be if it doesn't
# group neighborhoods in the same borough
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
toronto_df_dropna=toronto_df_dropna.sort_values(by=['PostalCode'])#to verify that it's identical
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
6,M1B,Scarborough,Malvern / Rouge
12,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
18,M1E,Scarborough,Guildwood / Morningside / West Hill
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae


In [9]:
# Changing the split Character from "/" to "," to match the requirement of the assignment
toronto_df_grouped['Neighborhood']=toronto_df_grouped['Neighborhood'].str.replace("/",",")
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## 3.3-Assigning "Not assigned" neighborhood the same value of Bourough

In [10]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
# Arranging the DataFrame to match the example given in the assignment
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df


,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill , Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford , Maryvale"
7,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands , ..."


### 3.4-Applying .shape method to print the number of rows of our dataframe

In [12]:
# print the number of rows of the cleaned dataframe
toronto_df_grouped.shape

(103, 3)

# Segmenting and Clustering Neighborhoods in Toronto Part 2

### 4.1 Loading the csv file given by Coursera

In [13]:
# load the coordinates from the given link on Coursera
coordinates = pd.read_csv('http://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
# rename the column "Postal Code" to "PostalCode" in order to match our dataframe
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## 4.2- Merging the table "toronto_df_grouped" from part 

In [15]:
# merge two table on the column "PostalCode"
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Checking if the results matches the example on Coursera

In [16]:
# create a new test dataframe
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford , Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands , ...",43.628947,-79.394420


# Segmenting and Clustering Neighborhoods in Toronto Part 3

### 5.1-Using geopy library to get the latitude and longitude values of Toronto

In [17]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [18]:
! pip install folium
import folium # map rendering library

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

#### Filtering Bouroughs withe word toronto

In [19]:
# filter borough names that contain the word Toronto
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [20]:
# create a new DataFrame with only boroughs that contain the word Toronto
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [21]:

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### 5.2- Using the Foursquare API to explore the neighborhoods

In [45]:
# The code was removed by Watson Studio for sharing.

In [43]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


#### Let's get the top 100 venues that are within a radius of 500 meters.

In [23]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [24]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1609, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


#### Checking how many venues were returned for each PostalCode

In [25]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West , Riverdale",43,43,43,43,43,43
M4L,East Toronto,"India Bazaar , The Beaches West",19,19,19,19,19,19
M4M,East Toronto,Studio District,40,40,40,40,40,40
M4N,Central Toronto,Lawrence Park,4,4,4,4,4,4
M4P,Central Toronto,Davisville North,7,7,7,7,7,7
M4R,Central Toronto,North Toronto West,21,21,21,21,21,21
M4S,Central Toronto,Davisville,36,36,36,36,36,36
M4T,Central Toronto,"Moore Park , Summerhill East",3,3,3,3,3,3


#### Finding out how many unique categories can be curated from all the returned venues

In [26]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 231 uniques categories.


In [27]:
venues_df['VenueCategory'].unique()[:30]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Greek Restaurant', 'Cosmetics Shop', 'Italian Restaurant',
       'Ice Cream Shop', 'Yoga Studio', 'Brewery',
       'Fruit & Vegetable Store', 'Pizza Place', 'Bookstore',
       'Restaurant', 'Dessert Shop', 'Juice Bar', 'Bubble Tea Shop',
       'Spa', 'Diner', 'Grocery Store', 'Furniture / Home Store', 'Café',
       'Coffee Shop', 'Bakery', 'Caribbean Restaurant',
       'Indian Restaurant', 'Japanese Restaurant', 'American Restaurant',
       'Frozen Yogurt Shop', 'Lounge'], dtype=object)

In [28]:
# one hot encoding
venues_df.rename(columns={'Neighborhood':'Neighborhoods'},inplace=True)#To avoid duplicates withe categories
toronto_onehot = venues_df[["PostalCode", "Borough", "Neighborhoods"]]  
dummies =pd.get_dummies(venues_df['VenueCategory'])
toronto_onehot = pd.concat([toronto_onehot,dummies], axis =1)

print(toronto_onehot.shape)
toronto_onehot.head()

(1609, 234)


,PostalCode,Borough,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West , Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Let's group rows by neighborhoods and by taking the mean of the frequency of occurrence of each category

In [29]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped.head()

(39, 234)


,PostalCode,Borough,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.000,0.0,0.000000
1,M4K,East Toronto,"The Danforth West , Riverdale",0.0,0.0,0.0,0.0,0.0,0.023256,0.0,...,0.0,0.0,0.023256,0.0,0.0,0.0,0.0,0.000,0.0,0.023256
2,M4L,East Toronto,"India Bazaar , The Beaches West",0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,0.000000
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.050000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.025,0.0,0.025000
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,0.000000


#### Let's create the new dataframe and display the top 10 venues for each PostalCode.

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, 3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]
    
# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Health Food Store,Pub,Trail,Neighborhood,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
1,M4K,East Toronto,"The Danforth West , Riverdale",Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Restaurant,Spa,Japanese Restaurant
2,M4L,East Toronto,"India Bazaar , The Beaches West",Pizza Place,Fast Food Restaurant,Sushi Restaurant,Brewery,Burrito Place,Liquor Store,Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery,Yoga Studio,Cheese Shop,Bookstore,Italian Restaurant
4,M4N,Central Toronto,Lawrence Park,Park,Lawyer,Bus Line,Swim School,Distribution Center,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
5,M4P,Central Toronto,Davisville North,Park,Hotel,Food & Drink Shop,Sandwich Place,Department Store,Breakfast Spot,Gym,Coworking Space,Doner Restaurant,Farmers Market
6,M4R,Central Toronto,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Chinese Restaurant,Fast Food Restaurant,Salon / Barbershop,Spa,Sporting Goods Shop,Restaurant,Rental Car Location
7,M4S,Central Toronto,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Gym,Café,Sushi Restaurant,Italian Restaurant,Coffee Shop,Indian Restaurant,Bar
8,M4T,Central Toronto,"Moore Park , Summerhill East",Park,Playground,Summer Camp,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
9,M4V,Central Toronto,"Summerhill West , Rathnelly , South Hill , For...",Pub,Coffee Shop,Sushi Restaurant,Restaurant,Fried Chicken Joint,Supermarket,Bagel Shop,Sports Bar,Pizza Place,Bank


## 5.3-Clustering neighborhood

#### Running K-means for clustering the TORONTO area into 5 Clusters

In [33]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 1, 1, 1, 1, 1, 3, 1], dtype=int32)

In [32]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_df_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head()

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,Neighborhood,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188,1,Greek Restaurant,Italian Restaurant,Coffee Shop,Bookstore,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Restaurant,Spa,Japanese Restaurant
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572,1,Pizza Place,Fast Food Restaurant,Sushi Restaurant,Brewery,Burrito Place,Liquor Store,Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery,Yoga Studio,Cheese Shop,Bookstore,Italian Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Lawyer,Bus Line,Swim School,Distribution Center,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [34]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,Neighborhood,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant
20,M5K,Downtown Toronto,"Toronto Dominion Centre , Design Exchange",43.647177,-79.381576,1,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Seafood Restaurant,American Restaurant,Salad Place,Gastropub,Steakhouse
21,M5L,Downtown Toronto,"Commerce Court , Victoria Hotel",43.648198,-79.379817,1,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Deli / Bodega
24,M5R,Central Toronto,"The Annex , North Midtown , Yorkville",43.672710,-79.405678,1,Sandwich Place,Café,Coffee Shop,Burger Joint,Pub,Indian Restaurant,Middle Eastern Restaurant,BBQ Joint,Pizza Place,Pharmacy
25,M5S,Downtown Toronto,"University of Toronto , Harbord",43.662696,-79.400049,1,Café,Bakery,Restaurant,Japanese Restaurant,Bookstore,Italian Restaurant,Bar,French Restaurant,Sandwich Place,Pub
26,M5T,Downtown Toronto,"Kensington Market , Chinatown , Grange Park",43.653206,-79.400049,1,Café,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Grocery Store,Gaming Cafe,Bar,Arts & Crafts Store,Bakery,Dessert Shop
27,M5V,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands , ...",43.628947,-79.394420,1,Airport Service,Airport Lounge,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Bar,Boat or Ferry,Boutique,Coffee Shop
28,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,1,Coffee Shop,Italian Restaurant,Seafood Restaurant,Café,Restaurant,Beer Bar,Japanese Restaurant,Hotel,Farmers Market,Bakery
29,M5X,Downtown Toronto,"First Canadian Place , Underground city",43.648429,-79.382280,1,Coffee Shop,Café,Restaurant,Hotel,Asian Restaurant,Seafood Restaurant,Japanese Restaurant,Deli / Bodega,Gym,Bar
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Gas Station,Candy Store,Restaurant,Diner,Baby Store,Athletics & Sports,Italian Restaurant


In [36]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##  Clusters details

#### Cluster 1

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,0,Health Food Store,Pub,Trail,Neighborhood,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Eastern European Restaurant


#### Cluster 2

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,"Toronto Dominion Centre , Design Exchange",1,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Seafood Restaurant,American Restaurant,Salad Place,Gastropub,Steakhouse
21,"Commerce Court , Victoria Hotel",1,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Deli / Bodega
24,"The Annex , North Midtown , Yorkville",1,Sandwich Place,Café,Coffee Shop,Burger Joint,Pub,Indian Restaurant,Middle Eastern Restaurant,BBQ Joint,Pizza Place,Pharmacy
25,"University of Toronto , Harbord",1,Café,Bakery,Restaurant,Japanese Restaurant,Bookstore,Italian Restaurant,Bar,French Restaurant,Sandwich Place,Pub
26,"Kensington Market , Chinatown , Grange Park",1,Café,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Grocery Store,Gaming Cafe,Bar,Arts & Crafts Store,Bakery,Dessert Shop
27,"CN Tower , King and Spadina , Railway Lands , ...",1,Airport Service,Airport Lounge,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Bar,Boat or Ferry,Boutique,Coffee Shop
28,Stn A PO Boxes,1,Coffee Shop,Italian Restaurant,Seafood Restaurant,Café,Restaurant,Beer Bar,Japanese Restaurant,Hotel,Farmers Market,Bakery
29,"First Canadian Place , Underground city",1,Coffee Shop,Café,Restaurant,Hotel,Asian Restaurant,Seafood Restaurant,Japanese Restaurant,Deli / Bodega,Gym,Bar
30,Christie,1,Grocery Store,Café,Park,Gas Station,Candy Store,Restaurant,Diner,Baby Store,Athletics & Sports,Italian Restaurant
31,"Dufferin , Dovercourt Village",1,Bakery,Pharmacy,Park,Gym / Fitness Center,Middle Eastern Restaurant,Music Venue,Recording Studio,Café,Brewery,Brazilian Restaurant


#### Cluster 3

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,2,Garden,Pool,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 4

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Moore Park , Summerhill East",3,Park,Playground,Summer Camp,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
10,Rosedale,3,Park,Playground,Trail,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 5

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Forest Hill North & West,4,Jewelry Store,Trail,Bus Line,Sushi Restaurant,Yoga Studio,Distribution Center,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


### Observation
##### Must of the neighborhoods are in cluster 2 wich the most common venues are coffee shops and restaurant